# Hola

In [5]:
!pip install torch --quiet

In [3]:
# Paso 2: Dataset pequeño de ejemplo
texto = """
La luna brilla sobre el mar.
Las estrellas miran en silencio.
La noche canta con voz serena.
"""

In [7]:
# Paso 3: Imports iniciales
import torch

In [9]:
# Paso 4: Tokenización
chars = sorted(list(set(texto)))
vocab_size = len(chars)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

data = torch.tensor(encode(texto), dtype=torch.long)

In [10]:
# Paso 5: Hiperparámetros
BLOCK_SIZE = 16
BATCH_SIZE = 4
EMBED_DIM = 32
N_HEADS = 2
N_LAYERS = 2
EPOCHS = 300
LEARNING_RATE = 1e-3

In [11]:
# Paso 6: Batching

def get_batch(data):
    ix = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,))
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in ix])
    return x, y

In [12]:
# Paso 7: Modelo MiniGPT
import torch.nn as nn
import torch.nn.functional as F

class SelfAttentionHead(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.query = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.value = nn.Linear(EMBED_DIM, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(BLOCK_SIZE, BLOCK_SIZE)))

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) / (C**0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        v = self.value(x)
        return wei @ v

class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([SelfAttentionHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_heads * head_size, EMBED_DIM)

    def forward(self, x):
        return self.proj(torch.cat([h(x) for h in self.heads], dim=-1))

class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(EMBED_DIM, 4 * EMBED_DIM),
            nn.ReLU(),
            nn.Linear(4 * EMBED_DIM, EMBED_DIM),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    def __init__(self):
        super().__init__()
        self.sa = MultiHeadAttention(N_HEADS, EMBED_DIM // N_HEADS)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(EMBED_DIM)
        self.ln2 = nn.LayerNorm(EMBED_DIM)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class MiniGPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_emb = nn.Embedding(vocab_size, EMBED_DIM)
        self.pos_emb = nn.Embedding(BLOCK_SIZE, EMBED_DIM)
        self.blocks = nn.Sequential(*[Block() for _ in range(N_LAYERS)])
        self.ln_f = nn.LayerNorm(EMBED_DIM)
        self.head = nn.Linear(EMBED_DIM, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_emb(idx)
        pos_emb = self.pos_emb(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.head(x)
        return logits

    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -BLOCK_SIZE:]
            logits = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            next_id = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, next_id), dim=1)
        return idx

model = MiniGPT()
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

In [13]:
# Paso 8: Entrenamiento
for step in range(EPOCHS):
    x, y = get_batch(data)
    logits = model(x)
    loss = F.cross_entropy(logits.view(-1, vocab_size), y.view(-1))

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 50 == 0:
        print(f"Paso {step} - Pérdida: {loss.item():.4f}")

Paso 0 - Pérdida: 3.2035
Paso 50 - Pérdida: 2.3259
Paso 100 - Pérdida: 1.7618
Paso 150 - Pérdida: 1.3205
Paso 200 - Pérdida: 0.9804
Paso 250 - Pérdida: 0.7618


In [18]:
# Paso 9: Generación de texto
context = torch.zeros((1, 1), dtype=torch.long)
print("\n--- Texto generado ---")
print(decode(model.generate(context, 100)[0].tolist()))


--- Texto generado ---

La noche sa che r.
Lanta czlcobre strenana el el ma mi
La.
Lanobrill ea sobrellel mhmase enastrellll
